In [1]:
%connect_info

{
  "shell_port": 52008,
  "iopub_port": 52009,
  "stdin_port": 52010,
  "control_port": 52012,
  "hb_port": 52011,
  "ip": "127.0.0.1",
  "key": "f94ec51c-fad7771b100dc99789562983",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-122f5d67-2e8a-4555-82d2-3ecf84cb11ea.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
%matplotlib inline

## Possible models

`bert-base-multilingual-cased`: (New, recommended) 12-layer, 768-hidden, 12-heads, 110M parameters. Trained on cased text in the top 104 languages with the largest Wikipedias

`xlm-mlm-100-1280`: 16-layer, 1280-hidden, 16-heads XLM model trained with MLM (Masked Language Modeling) on 100 languages.

`distilbert-base-multilingual-cased`: 6-layer, 768-hidden, 12-heads, 134M parameters The multilingual DistilBERT model distilled from the Multilingual BERT model bert-base-multilingual-cased checkpoint.

In [3]:
import sys
sys.argv = sys.argv[:1]

In [4]:
import os
import pickle
import numpy as np
import tensorflow as tf

from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from evaluation import  evaluate, MetricsReporterCallback
from utils import build_model_name, convert_flags_to_dict, define_cnn_flags
from preprocess import load_and_tokenize_dataset

from transformers import AutoTokenizer, TFAutoModel

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [5]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [6]:
SEED = 42
BASE_DIR = os.path.expanduser("~")     # this will point to the user's home
TRAIN_DIR = "ray_results"

FLAGS = define_cnn_flags(tf.compat.v1.flags, BASE_DIR, TRAIN_DIR)

In [7]:
model_type = 'distilbert-base-multilingual-cased'

In [8]:
# tokenizer = AutoTokenizer.from_pretrained(model_type)
# transformer = TFAutoModel.from_pretrained(model_type)

In [9]:
FLAGS.conv_layers = [int(i) for i in FLAGS.conv_layers]
FLAGS.dense_layers = [int(i) for i in FLAGS.dense_layers]

_config = convert_flags_to_dict(FLAGS)
_config["codes"] = (["DE', 'GA', 'HI', 'PT', 'ZH"]
                    if FLAGS.language_code is 'all' else [FLAGS.language_code])

cwd = os.getcwd()

RESULTS = {}